In [21]:
from cnn_method import *
from cnn_preproc_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os

raw_data_file =  "./DATA/PRICE_LIQUIDASSET_60_MIN.csv"
TARGET_TO_PREDICT = "EURUSD"
FUTURE_PERIOD_PREDICT = 1

TARGET_FUNCTION = "cumulative_returns"
TARGET_THRESHOLD = 0.001
BREAKOUT_WINDOW = 60

In [22]:
def create_target_2(df, target_col, FUTURE_PERIOD_PREDICT, TARGET_FUNCTION = "cumulative_returns", keras_preproc = True):
    if TARGET_FUNCTION == "cumulative_returns":
        TARGET_FUNCTION_R = cumulative_returns
    elif TARGET_FUNCTION == "mod_sharpe":
        TARGET_FUNCTION_R = mod_sharpe

    df.loc[:,'target'] = df[target_col].rolling(window = FUTURE_PERIOD_PREDICT).apply(lambda x: TARGET_FUNCTION_R(x))
    df.loc[:,'target'] = df['target'].shift(-FUTURE_PERIOD_PREDICT+1)
    df = df.dropna()
    return df


def clean_data_breakout_x(df, target_col, breakout_window = 60):
    price_df = df.fillna(method = "ffill").dropna()
    return_df = df.pct_change()

    return_df = filter_off_trading_day(return_df, target_col)
    filtered_index = return_df.index
    price_df = price_df.reindex(filtered_index)
    x_df = price_df.rolling(window = breakout_window).apply(lambda x: breakout(x)*1,raw = False)
    return_df["target"] = return_df[target_col]
    fdf = pd.merge(x_df, return_df[["target"]], left_index = True, right_index = True)
    fdf.dropna()

    return fdf

In [23]:
df = load_data(raw_data_file)


Load Data: Done!


In [24]:
df = clean_data_breakout_x(df, target_col = TARGET_TO_PREDICT, breakout_window = BREAKOUT_WINDOW)


In [25]:
df = create_target_2(df, "target", FUTURE_PERIOD_PREDICT, TARGET_FUNCTION)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


In [26]:
FLIP = False
df = classify_target(df, "target", TARGET_THRESHOLD, FLIP)


In [27]:

def split_df(df, end_split):
    #Split df and get index
    start_index, end_index = get_index_from_date(df, end_split)
    target_col= "target"
    x_columns = [j for j in df.columns if j != target_col]

    #Scaling
    # scaler = sklearn.preprocessing.MinMaxScaler(feature_range = (0,1))
    scaler = sklearn.preprocessing.StandardScaler()
    #Fit train_x
    train_x_data = df[x_columns].iloc[start_index[0]:(end_index[0]+1)].values
    scaler.fit(train_x_data)
    #Scale all
    df.loc[:,x_columns] = scaler.transform(df[x_columns])
    X = df[x_columns].values
    Y = df[target_col].values

    return df, X, Y, start_index, end_index, scaler

import datetime
end_split = [datetime.datetime(2011,1,1), datetime.datetime(2013,1,1)]




In [28]:
df, X, Y, start_index, end_index, scaler = split_df(df, end_split)


In [29]:
train_data_gen, val_data_gen, test_1_data_gen, test_2_data_gen, shape_x = TSGenerator(X, Y, SEQ_LEN, BATCH_SIZE, start_index, end_index)


NameError: name 'SEQ_LEN' is not defined

In [30]:
class_weights = get_class_weights(df, start_index, end_index)



[0.62885337 2.44018987]


In [31]:
df.columns


Index(['EURUSD', 'EEM', 'EFA', 'EWZ', 'FXI', 'GDX', 'HYG', 'IAU', 'IWM', 'SPY',
       'USO', 'VWO', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XOP', 'JNK',
       'IYR', 'VEA', 'SLV', 'XLV', 'RSX', 'TLT', 'EWJ', 'OIH', 'GLD', 'EZU',
       'KRE', 'SMH', 'XLB', 'XRT', 'LQD', 'EWT', 'XLY', 'VNQ', 'EWH', 'EWW',
       'XBI', 'DIA', 'EWG', 'VGK', 'IEF', 'EMB', 'FEZ', 'AGG', 'ITB', 'EWC',
       'UNG', 'USDJPY', 'GBPUSD', 'AUDUSD', 'USDCAD', 'EURJPY', 'NZDUSD',
       'XAGUSD', 'target'],
      dtype='object')

In [33]:
np.mean(df.iloc[start_index[1]:end_index[1]]["target"])

0.14956786171574904